In [ ]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import cv2
import glob
from sklearn.metrics import classification_report

from __future__ import print_function, division
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch
from torch import nn
import torchvision 
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import models
from torchvision import transforms
from torchsummary import summary


cudnn.benchmark = True
plt.ion() 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/rnn/train.csv')

In [ ]:
df.head

<bound method NDFrame.head of       tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288585          -0.020294          -0.132905   
1              0.278419          -0.016411          -0.123520   
2              0.279653          -0.019467          -0.113462   
3              0.279174          -0.026201          -0.123283   
4              0.276629          -0.016570          -0.115362   
...                 ...                ...                ...   
7347           0.299665          -0.057193          -0.181233   
7348           0.273853          -0.007749          -0.147468   
7349           0.273387          -0.017011          -0.045022   
7350           0.289654          -0.018843          -0.158281   
7351           0.351503          -0.012423          -0.203867   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.995279         -0.983111         -0.913526         -0.995112   
1            -0.998245       

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [ ]:
df['Activity'].unique()

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [ ]:
X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/rnn/test.csv')

In [ ]:
X_test = np.array(df_test.iloc[:, :-1])
y_test = np.array(df_test.iloc[:, -1])


In [ ]:
X.shape

(7352, 562)

In [ ]:
len(X)

7352

# Prepare data

In [ ]:
from sklearn import preprocessing
encoder=preprocessing.LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)
y.shape

(7352,)

In [ ]:
from sklearn import preprocessing
encoder=preprocessing.LabelEncoder()
encoder.fit(y_test)
y_test=encoder.transform(y_test)
y_test.shape

(2947,)

In [ ]:
train_transforms = transforms.Compose([
  transforms.ToTensor()
])
class myDataset(Dataset):
  def __init__(self, X, label, trans = None):
    self.X = X
    self.label = label
    self.trans = trans
  def __len__(self):
    return len(self.X)
  def __getitem__(self,index):
    x = np.array([self.X[index]])
    lb = self.label[index]
    if self.trans:
      x = self.trans(x)
    return x, lb


In [ ]:
def prepare_loader(datasets):
  dataloader = DataLoader(dataset=datasets, batch_size= 1000, shuffle=True, num_workers=2)
  return dataloader 

In [ ]:
train_dataset = myDataset(X, y)
test_dataset = myDataset(X_test, y_test)

In [ ]:
a, b = train_dataset[0]

In [ ]:
a.shape

(1, 562)

In [ ]:
train_loader = prepare_loader(train_dataset)
test_loader = prepare_loader(test_dataset)

In [ ]:
sq, lb= iter(train_loader).next()
sq.shape

torch.Size([1000, 1, 562])

In [ ]:
sq[0].shape

torch.Size([1, 562])

#model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 6
num_epochs = 1000
batch_size = 1000
learning_rate = 0.01

input_size = 562
sequence_length = 1
hidden_size = 128
num_layers = 4
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 1, 562), h0: (1, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 1, 128)
        
        # Decode the hidden state of the last time step

        out = out[:, -1, :]

        # out: (n, 128)
         
        out = self.fc(out)

        # out: (n, 6)
        return out



In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
sq = sq.float().to(device)
out = model(sq.float())

In [ ]:
out.size()

torch.Size([1000, 6])

In [ ]:
out

tensor([[ 0.0947,  0.0044, -0.0537, -0.0438,  0.0040,  0.0646],
        [ 0.0948,  0.0041, -0.0535, -0.0437,  0.0040,  0.0644],
        [ 0.0949,  0.0042, -0.0535, -0.0438,  0.0041,  0.0642],
        ...,
        [ 0.0947,  0.0045, -0.0536, -0.0439,  0.0036,  0.0647],
        [ 0.0949,  0.0042, -0.0535, -0.0439,  0.0042,  0.0641],
        [ 0.0950,  0.0043, -0.0536, -0.0441,  0.0042,  0.0641]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
te = torch.max(out.data, 1)
print(te[1])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
n_total_steps = len(train_loader)
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 2
  for i, (xx, labels) in enumerate(train_loader):  
    # input: [1000 1, 562]
    xx = xx.float().to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = model(xx)
    loss = loss_func(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * xx.size(0)
    if i % reporting_steps == reporting_steps-1:
        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], ave_Loss: {running_loss/reporting_steps:.4f}')
        running_loss = 0.0
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for i, (xx, labels) in enumerate(loader):
      xx, labels = xx.float().to(device), labels.to(device)
      outputs = model(xx)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue


In [ ]:
a = [0,1,2,3]
b = [1,1,2,1]
ab = (a == b)
ab

False

In [ ]:
# Loss and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

# Train the model
for epoch in range(num_epochs):
  train_epoch(epoch, model, train_loader, loss_func, optimizer, device)
  ypred, ytrue = test_epoch(epoch, model, test_loader, device)
  print(classification_report(ytrue, ypred))


Epoch [1/1000], Step [2/8], ave_Loss: 1793.0584
Epoch [1/1000], Step [4/8], ave_Loss: 1792.8596
Epoch [1/1000], Step [6/8], ave_Loss: 1793.3241
Epoch [1/1000], Step [8/8], ave_Loss: 1212.3265
              precision    recall  f1-score   support

           0       0.18      1.00      0.31       537
           1       0.00      0.00      0.00       491
           2       0.00      0.00      0.00       532
           3       0.00      0.00      0.00       496
           4       0.00      0.00      0.00       420
           5       0.00      0.00      0.00       471

    accuracy                           0.18      2947
   macro avg       0.03      0.17      0.05      2947
weighted avg       0.03      0.18      0.06      2947



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [2/1000], Step [2/8], ave_Loss: 1792.7228
Epoch [2/1000], Step [4/8], ave_Loss: 1792.1175
Epoch [2/1000], Step [6/8], ave_Loss: 1792.3034
Epoch [2/1000], Step [8/8], ave_Loss: 1209.8177
              precision    recall  f1-score   support

           0       0.18      1.00      0.31       537
           1       0.00      0.00      0.00       491
           2       0.00      0.00      0.00       532
           3       0.00      0.00      0.00       496
           4       0.00      0.00      0.00       420
           5       0.00      0.00      0.00       471

    accuracy                           0.18      2947
   macro avg       0.03      0.17      0.05      2947
weighted avg       0.03      0.18      0.06      2947



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [3/1000], Step [2/8], ave_Loss: 1790.4342
Epoch [3/1000], Step [4/8], ave_Loss: 1791.5212
Epoch [3/1000], Step [6/8], ave_Loss: 1789.0648
Epoch [3/1000], Step [8/8], ave_Loss: 1209.4528
              precision    recall  f1-score   support

           0       0.18      1.00      0.31       537
           1       0.00      0.00      0.00       491
           2       0.00      0.00      0.00       532
           3       0.00      0.00      0.00       496
           4       0.00      0.00      0.00       420
           5       0.00      0.00      0.00       471

    accuracy                           0.18      2947
   macro avg       0.03      0.17      0.05      2947
weighted avg       0.03      0.18      0.06      2947



...

Epoch [838/1000], Step [2/8], ave_Loss: 625.1595
Epoch [838/1000], Step [4/8], ave_Loss: 649.1822
Epoch [838/1000], Step [6/8], ave_Loss: 624.5707
Epoch [838/1000], Step [8/8], ave_Loss: 412.5112
              precision    recall  f1-score   support

           0       0.78      1.00      0.87       537
           1       0.00      0.00      0.00       491
           2       0.61      0.98      0.75       532
           3       0.78      0.91      0.84       496
           4       0.93      0.83      0.87       420
           5       0.94      0.87      0.91       471

    accuracy                           0.77      2947
   macro avg       0.67      0.77      0.71      2947
weighted avg       0.66      0.77      0.71      2947



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [839/1000], Step [2/8], ave_Loss: 618.6723
Epoch [839/1000], Step [4/8], ave_Loss: 613.9064
Epoch [839/1000], Step [6/8], ave_Loss: 621.7901
Epoch [839/1000], Step [8/8], ave_Loss: 413.2933
              precision    recall  f1-score   support

           0       0.79      1.00      0.88       537
           1       0.00      0.00      0.00       491
           2       0.60      0.98      0.74       532
           3       0.77      0.91      0.83       496
           4       0.91      0.86      0.88       420
           5       0.96      0.84      0.89       471

    accuracy                           0.77      2947
   macro avg       0.67      0.76      0.71      2947
weighted avg       0.66      0.77      0.70      2947



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [840/1000], Step [2/8], ave_Loss: 596.8140
Epoch [840/1000], Step [4/8], ave_Loss: 623.3397
Epoch [840/1000], Step [6/8], ave_Loss: 616.1143
Epoch [840/1000], Step [8/8], ave_Loss: 403.8558
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       537
           1       1.00      0.01      0.02       491
           2       0.59      0.98      0.73       532
           3       0.76      0.91      0.83       496
           4       0.92      0.86      0.89       420
           5       0.96      0.82      0.88       471

    accuracy                           0.77      2947
   macro avg       0.84      0.76      0.71      2947
weighted avg       0.83      0.77      0.71      2947

Epoch [841/1000], Step [2/8], ave_Loss: 596.8876
Epoch [841/1000], Step [4/8], ave_Loss: 614.9181
Epoch [841/1000], Step [6/8], ave_Loss: 582.1303
Epoch [841/1000], Step [8/8], ave_Loss: 404.9765
              precision    recall  f1-score   support

          

In [ ]:
with torch.no_grad():
  correct = 0
  sum = 0
  for input, labels in test_loader:
    labels = labels.to(device)
    output = model(input.float().to(device))
    _, predicted = torch.max(output.data, 1)
    sum += labels.size(0)
    correct += (predicted == labels).sum().item()
  acc = 100.0 * correct / sum
  print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 93.55276552426196 %
